The input of the NN is 1D. 

In [45]:
import numpy as np
import torch
import torch.nn as nn
import math
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

INPUT_SIZE = 1
OUTPUT_SIZE = 1

LOWER_BOUND = -10
UPPER_BOUND = 10

def quad_fn(a, b, c, x):
	return a*x**2 + b*x + c

def x_square(x: torch.tensor) -> torch.Tensor:
	return x**2

# DEFINE GIVEN FUNCTION V(x): ax^2 + bx + c
given_fn = x_square

In [107]:
# CUSTOM LOSS FUNCTION:
def epsilon_Loss(v_x, model_u, lower_bound, upper_bound, n_points):
    """
    GOAL: Epsilon function evaluated at u using discretized estimation
    minimizing Epsilon(u) = 
    
    ARGS: 
    n_points (int): number of discretized points on the interval [-L, L]
    e.g.: -(L)|---|---|---|---|(L) interval has n_points = 5

    v_x (torch.Tensor): function instance
    model_u (torch.Tensor): model output
    """
    sum = 0
    discrete_points = np.linspace(lower_bound, upper_bound, n_points)
    for i in discrete_points:
        x_i = torch.tensor([i], requires_grad=True, dtype=torch.float)
        u_xi = model_u(x_i)
        u_xi.backward()
        u_prime = x_i.grad
        v_xi = v_x(x_i)
        sum += torch.abs(u_prime)**2 + v_xi*(u_xi**2)

    return 0.5*sum

# NORMALIZE MODEL u(x) OUTPUT:
def normalize_u(model_u, lower_bound, upper_bound, n_points):
    discrete_points = np.linspace(lower_bound, upper_bound, n_points)
    h = discrete_points[1] - discrete_points[0]
    s = 0
    for i in discrete_points:
        x_i = torch.tensor([i], requires_grad=True, dtype=torch.float)
        t = model_u(x_i)**2
        s += t
    denom = torch.sqrt(h) * torch.sqrt(s)

    return 1/denom

In [96]:
p = [-10.,  -5.,   0.,   5.,  10.]
t = torch.tensor([p[0]], requires_grad=True)
Q = t**2
Q.backward()

In [105]:
epsilon_Loss(5, given_fn, model_u, -10, 10)

tensor([36.7810], grad_fn=<MulBackward0>)

In [106]:
v_x = given_fn
l_b = -10
u_b = 10
n_points = 5
model_u = Nonlinear_2(20)

sum = 0
d_points = np.linspace(l_b, u_b, n_points)
for i in d_points:
    x_i = torch.tensor([i], requires_grad=True, dtype=torch.float)
    print("x_i = " + str(x_i))
    u_xi = model_u(x_i)
    print("u_xi = " + str(u_xi))
    u_xi.backward()
    u_prime = x_i.grad
    print("u_prime = " + str(u_prime))
    v_xi = v_x(x_i)
    print("v_xi = " + str(v_xi))
    foo = torch.abs(u_prime)**2 + v_xi*(u_xi**2)
    print(foo)
    print('-----')
    sum += foo
print(sum)
    

x_i = tensor([-10.], requires_grad=True)
u_xi = tensor([0.0161], grad_fn=<AddBackward0>)
u_prime = tensor([-0.0025])
v_xi = tensor([100.], grad_fn=<PowBackward0>)
tensor([0.0260], grad_fn=<AddBackward0>)
-----
x_i = tensor([-5.], requires_grad=True)
u_xi = tensor([-0.0088], grad_fn=<AddBackward0>)
u_prime = tensor([-0.0100])
v_xi = tensor([25.], grad_fn=<PowBackward0>)
tensor([0.0020], grad_fn=<AddBackward0>)
-----
x_i = tensor([0.], requires_grad=True)
u_xi = tensor([-0.0283], grad_fn=<AddBackward0>)
u_prime = tensor([0.0047])
v_xi = tensor([0.], grad_fn=<PowBackward0>)
tensor([2.2252e-05], grad_fn=<AddBackward0>)
-----
x_i = tensor([5.], requires_grad=True)
u_xi = tensor([-0.0865], grad_fn=<AddBackward0>)
u_prime = tensor([-0.0065])
v_xi = tensor([25.], grad_fn=<PowBackward0>)
tensor([0.1872], grad_fn=<AddBackward0>)
-----
x_i = tensor([10.], requires_grad=True)
u_xi = tensor([-0.1009], grad_fn=<AddBackward0>)
u_prime = tensor([-0.0014])
v_xi = tensor([100.], grad_fn=<PowBackward0>)


In [14]:
# CREATING DATASET:
x_values = [i for i in np.arange(LOWER_BOUND, UPPER_BOUND, math.pi/50)]
y_values = [given_fn(i) for i in x_values]

x_train, x_test=train_test_split(x_values,test_size=0.2)
y_train = [given_fn(i) for i in x_train]
y_test = [given_fn(i) for i in x_test]

x_train = np.array(x_train, dtype=np.float32).reshape(-1, 1)
y_train = np.array(y_train, dtype=np.float32).reshape(-1, 1)

x_test = np.array(x_test, dtype=np.float32).reshape(-1, 1)
y_test = np.array(y_test, dtype=np.float32).reshape(-1, 1)

print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(80, 1) (80, 1) (20, 1) (20, 1)


In [8]:
# PLOT DATA
def plot_performance(x_train, y_train, x_test, predicted=None):
    plt.clf()
    plt.figure(figsize=(8, 6), dpi=80)
    plt.plot(x_train, y_train, 'go', label='True data', alpha=0.5)
    if predicted != None:
        plt.plot(x_test, predicted, '--', label='Predictions', alpha=0.5)
    plt.legend(loc='best')
    plt.show()

In [54]:
# CREATING MODEL CLASS
class Nonlinear_2(nn.Module):
    def __init__(self, n):
        # One hidden layer with n nodes
        super().__init__()
        self.hidden = nn.Linear(1, n)
        self.output = nn.Linear(n, 1)
        
        self.sigmoid = nn.Sigmoid()
        self.tanh = nn.Tanh()
    
    def forward(self, x, use_tanh_fn = False, activation_on_output = False):
        if use_tanh_fn == True:
            x = self.hidden(x)
            x = self.tanh(x)
            x = self.output(x)
            
        else:
            x = self.hidden(x)
            x = self.sigmoid(x)
            x = self.output(x)
            
        if activation_on_output == False:
            return x
        else:
            if use_tanh_fn == True:
                return self.tanh(x)
            else:       
                return self.sigmoid(x)
        # output is a linear combination of the hidden layers because 
        # we perform regression ???
        return x

In [57]:
# DEFINE HYPER-PARAMETERS
batch_size = 50
learningRate = 0.05
num_epochs = 1000
# num_epochs = int(num_iters/(len(x_train)/batch_size))

#INIT PARAMETERS: 
v_x = given_fn
l_b = -10
u_b = 10
n_points = 5

#INIT MODEL
model = Nonlinear_2(20)
if torch.cuda.is_available():
    model.cuda()

# INIT OPTIMIZER CLASS
# What is an optimizer: 
# SGD:
# SGD_optimizer = torch.optim.SGD(model.parameters(), lr=learningRate)
# Adam:
adam_optimizer = torch.optim.Adam(model.parameters(), lr=0.learningRate, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

# INIT LOSS FUNCTION: MSE
criterion = torch.nn.MSELoss()

In [58]:
# TRANING MODEL
x_epochs = [i for i in range(num_epochs)]
y_loss = []
for epoch in range(num_epochs):
    for (input, output) in (zip(x_train, y_train)):
        # if torch.cuda.is_available():
        #     input = Variable(torch.from_numpy(input).cuda())
        #     output = Variable(torch.from_numpy(output).cuda())
        # else:
        input = Variable(torch.from_numpy(input))
        output = Variable(torch.from_numpy(output))
    
        adam_optimizer.zero_grad()
        res = model(input)
        loss = criterion(res, output)
        loss.backward()
        adam_optimizer.step()
    y_loss.append(loss)
    if epoch % 50 == 0:
        print('epoch {}, loss {}'.format(epoch, loss.item()))

Nonlinear_2(
  (hidden): Linear(in_features=1, out_features=20, bias=True)
  (output): Linear(in_features=20, out_features=1, bias=True)
  (sigmoid): Sigmoid()
  (tanh): Tanh()
)